# 東森新聞雲爬蟲練習
## 練習從東森新聞雲網站中，爬取新聞摘要及詳細內容。
## 學習利用Selenium模擬人為操作，更新動態網頁後爬取新聞內容。

In [1]:
import requests
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

### 以財經新聞為例，先連結到財經新聞網頁，複製其URL。

In [2]:
ETtoday_url = "https://www.ettoday.net/news/focus/%E8%B2%A1%E7%B6%93/"  #財經新聞

### 用Selenium打開一個瀏覽器，連結到網站。然後模擬下拉網頁的動作，讓網頁完成更新，再處理後續動作。

In [11]:
browser = webdriver.Chrome(executable_path="./chromedriver.exe")
browser.get(ETtoday_url)  # 打開瀏覽器並連到東森新聞雲網頁

SCROLL_PAUSE_TIME = 1

#
# 以下是用Selenium模擬下拉網頁動作，讓網頁更新
#
last_height = browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")

while True:
    '''
    Your code here
    
    提示：可參考以下的Stack Overflow: 
    https://stackoverflow.com/questions/48850974/selenium-scroll-to-end-of-page-indynamically-loading-webpage/48851166
    '''
    # Scroll down to  the bottom.
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    
    # Wait to load the page
    time.sleep(SCROLL_PAUSE_TIME)
    
    # Calculate new scroll height and compare with last scroll height.
    new_height = browser.execute_script("return document.body.scrollHeight")
    
    if new_height == last_height:
        print("到達頁面底端")
        break
    last_height = new_height
    print("網頁更新中")

網頁更新中
網頁更新中
網頁更新中
網頁更新中
網頁更新中
網頁更新中
網頁更新中
網頁更新中
網頁更新中
到達頁面底端


### 到這裡網頁已經更新完畢，所有的新聞都已經出現在網頁上。接下來做爬取收集新聞的動作。

In [12]:
#
# 這裡先建立一個函數，其功能是連到外部連結，並爬取新聞內容。
#
def getNewsDetailContent(link_url):
    resp = requests.get(link_url)
    resp.encoding = 'utf-8'
    #print(resp.text)

    soup = BeautifulSoup(resp.text, 'lxml')
    news_content = soup.find("div", attrs={'class':'story'}).find_all("p")
    for p in news_content:
        """
        .string屬性說明：
        (1) 若當前tag節點底下沒有其他tag子節點，會直接抓取內容(返回"NavigableString")
        (2) 若當前tag節點底下只有唯一的一個tag子節點，也會直接抓取tag子節點的內容(返回"NavigableString")
        (3) 但若當前tag節點底下還有很多個tag子節點，.string就無法判斷，(返回"None")
        """
        if ((p.string) is not None):
            print(p.string)

### 解析HTML並萃取新聞摘要，若有外部的連結，再連到外部連結並把詳細新聞內容抓取下來。

In [13]:
# 爬取網頁內容，解析後萃取新聞摘要
html = browser.page_source
soup = BeautifulSoup(html, "lxml")
all_news = soup.find("div", attrs={'class':'block block_1 infinite_scroll'})

news_block = all_news.find_all('div', attrs={'class':'piece clearfix'})

for i, news_item in enumerate(news_block):
    print("----------------------------------------------------------------------")
    news_body = news_item.find('h3')
    print("\n[%d] %s\n" % (i, news_body.a.string))
    
    # 
    # 連到外部連結，擷取詳細新聞內容
    #
    externalLink = "https://www.ettoday.net" + news_body.a["href"]
    getNewsDetailContent(externalLink)


----------------------------------------------------------------------

[0] 挖角大立光工程師竊密案　先進光電前總經理等6人遭判刑

記者游瓊華／台中報導
股王大立光爆內鬼案，中檢偵辦查出先進光電科技公司前羅姓總經理10年前從大立光挖角多位工程師，還把大立光技術帶到先進光電發展製程。台中地院26日依照違反著作權法等罪將羅姓總經理及工程師等6人，各判刑6到5月徒刑；先進光電遭判侵害大立光著作財產權，科罰金新台幣60萬元。
民事部分，先進光電侵害大立光點膠針頭結構、遮光片送料機構，經智慧財產法院審理後，先進光電在一審被判賠15億2247萬多元，目前上訴二審中。
中檢起訴指出，先進光透過羅姓總經理，從99年開始為發展新一波自動化生產進程，陸續挖走大立光鄒姓、翁姓、謝姓以及朱姓、張姓以及徐姓等工程師，違反當初與大立光簽訂的聘僱合約書中條約，離職2年內，不得侵害大立光的商業秘密及智財權等。
台中地檢署檢察官指揮偵辦，發現鄒姓工程師等人將大立光電磁紀錄帶到先進光電，作為研發參考之用，將鄒姓工程師等7人與先進光電依違反著作權法、妨害電腦祕密等罪起訴。台中地院審理後，法官認為鄒姓工程師等人侵害大立光著作財產權，予以判刑。
台中地院判處羅姓總經理、鄒姓、翁姓、謝姓與朱姓等5名工程師各6月徒刑，可易科罰金；先進光電科罰金新台幣60萬元；徐姓工程師依背信罪判處5月徒刑，可易科罰金；張姓工程師因未於在職期間蒐集技術資料，判無罪。全案可上訴。
【更多新聞】
保持溫度冬季必備
----------------------------------------------------------------------

[1] 台股受累3因素走跌　金管會：亞股走弱、美國紓困案、全球疫情

▲金管會信心喊話，台股基本面仍然良好。（圖／記者陳依旻攝）
記者陳依旻／台北報導
台股今 （26）日重挫 287 點，創今年以來最大跌點，金管會證期局副局長蔡麗玲分析，主要是亞股疲弱、美國紓困案不確定，以及全球疫情變化等3因素衝擊。
蔡麗玲進一步說明，台股雖然下跌，但成交量仍很大，上市股票成交量達 3526.25 億元，「且基本面也良好」，截至去年 12 月底，台股本益比 22.37 倍，現金殖利率 2.81%，加計股票股利則有 2.99

過去十年，投資0050跟0056，報酬率會相差來到125%之多。那定期定額呢？
▲0050、0056定期定額報酬率，可點圖放大。（圖／翻攝自Jet Lee的投資隨筆）
就算你很有紀律的持續投入，股息有再投入，複利滾動，但因為選擇錯誤策略，表面上看來有賺到錢，但事實上遠遠落後一大段。
第二種失誤，選錯市場。
在2011年以前，如果你選擇新興市場來做為定期定額標的，我相信是很符合人性的選擇。
▲新興市場作為定期定額標的的報酬率，可點圖放大。（圖／翻攝自Jet Lee的投資隨筆）
但如果你實際真的這麼做，會得到怎樣的結果？
▲新興市場作為定期定額標的，總報酬將落後於定期定額美國大盤，可點圖放大。（圖／翻攝自Jet Lee的投資隨筆）
第三種失誤，花大錢做投資。
在現今一百多檔台股一般股票型基金裡，0050的十年績效可以排到前二十名。
▲0050十年績效相當不錯，可點圖放大。（圖／翻攝自Jet Lee的投資隨筆）
但如果你去看定期定額的報酬率，0050的定期定額報酬率其實可以到前十名。
▲0050定期定額報酬率可到前十。（圖／翻攝自Jet Lee的投資隨筆）
投資這些共同基金，你每年要花的費用肯定比ETF要高很多，但真的都有幫助投資人賺到應賺的錢嗎？
這一百多檔的基金裡，有超過3成是沒有十年績效的，代表有許多基金在過去十年裡消失，又有許多高費用共同基金成立。那當初花大錢定期定額投資那些「下市基金」的投資人呢？他們還會覺得定期定額就是投資獲利的保證？
「所以定期定額就是基金投資的必勝法門？」我看未必。
選錯投資策略、選錯投資市場、選錯投資工具，不管你再怎麼有紀律，頂多只能得到一個不上不下的報酬。
或許短期的績效無法讓你跟人比擬，但長期累積下來的財富會讓你很有感。
▲選對ETF，定期定額會很有感覺，可點圖放大。（圖／翻攝自Jet Lee的投資隨筆）
請記得，定期定額很好，但用對地方會對你更有幫助。
----------------------------------------------------------------------

[15] 聯電將調薪！幅度約2.5~6%　助理工程師加薪1000元

▲聯電。（圖／聯電提供）
記者姚惠茹／台北報導
晶圓代工廠聯電（2303）今（26）日證實，針對市場傳出即將為員工調薪一事，強調公司內部的確有針對員工薪水進行調整，而且這部

▲旱災中央災害應變中心今（26）日再度召開會議，經濟部政務次長曾文生(圖左)呼籲全民節約用水。（圖／記者林淑慧攝）
記者林淑慧／台北報導
年前水情持續吃緊，旱災中央災害應變中心今（26）日再度召開會議，經濟部政務次長曾文生表示，由於竹苗地區水庫蓄水量均不到三成，經濟部等新增抗旱水源約33.6萬噸；不過，預估二、三月降雨量仍偏少，桃園以南仍須持續開源節流。
針對農地供灌策略，旱災中央災害應變中心已經陸續宣布今年一期作嘉南、台中、新竹、苗栗停灌共約4.7萬公頃，以及桃園、觀音、大園及石門灌區，總計2.8萬公頃農地一期稻作停灌。
除了農業停灌外，新竹、苗栗及臺中地區也已實施減量供水橙燈，每月1,000度以上工業大用水戶節水7%，非工業大用水戶及游泳池、洗車、三溫暖、水療業者及其他非生活必需之用水節水10%，並視水情狀況循序漸進至20%。
旱災中央災害應變中心表示，台水公司針對減量供水地區，持續每周抄表查核，針對未達節水目標的工業用戶，將要求主管機關發函勸導，以落實查核管控作業。
----------------------------------------------------------------------

[28] 陸央行委員：應取消GDP增長目標　避免地方靠借錢拉數據

記者蔡紹堅／綜合報導
大陸央行中國人民銀行貨幣政策委員會委員馬駿日前表示，大陸應從今年起永久取消GDP增長目標，把穩定就業與控制通膨作為宏觀政策的主要目標，「地方習慣性層層加碼，把GDP目標定得很高，從而加大隱性債務的金融風險，因為靠借錢來實現投資拉動GDP比其它辦法都容易。」
馬駿指出，GDP的數字可以作為一個預測，財政部門可以將其作為一個基礎來進行財政收支的預測，投資部門可以將其作為基礎來預測投資行為，但不應該將其作為考核地方政府官員業績的指標。
馬駿提到，2020年初的時候，因為疫情帶來巨大不確定性，無法定GDP增速目標，「定得太高，做不到，但定得太低，又起不到提振信心的作用，所以沒有必要設，去年政府工作報告決定2020年不設當年的GDP增長目標，是非常正確的。」
馬駿認為，繼續設立GDP增速目標會出現一些問題，「地方習慣性層層加碼，把地方的GDP目標定得很高，從而加大隱性債務的金融風險，因為靠借錢來實現投資拉動GDP比其它辦法都容易，強調GDP考核的情況下，不可避免地會出現一些地方

▲google地圖在美顯示新冠肺炎疫苗接種位置。（圖／取自9to5google）
記者陳俐穎／綜合報導
Google稍早宣布，在Google Maps與搜尋服務上，讓更多人可以快速知道新冠肺炎疫苗接種位置，也釋出Google所屬的建築物，做為疫苗接種的場所，但此服務目前僅在美國使用。
報導指出，現在用戶在Google上進行搜索時，會出現「COVID-19疫苗」的面板，面板中分為多個部分，包含新聞與安全資訊，而點下「從哪裡獲得」將顯示地圖並顯示疫苗接種位置。
Google也宣布，將捐助1.5憶美元，折合台幣41億元，提供美國疾病控制中心基金會、世界衛生組織，以及其他非營利組織廣告版面，而其中5000萬美元，則用做向偏遠地區提供疫苗接種資訊的經費。
----------------------------------------------------------------------

[39] 薪水會全數交給另一半嗎？　網列「1條件」狂點頭：我很樂意

▲有些人會將錢全部交給另一半管理。示意圖，與本文無關。（圖／記者李佳蓉攝）
記者曾筠淇／綜合報導
每對情侶、夫妻對金錢的想法都不盡相同，一名網友發文表示，如果另一半要求自己將薪水全數交出，「大家可以接受嗎？」貼文曝光後，立刻引發熱議，有網友就回應，「說真的，管錢很累耶」、「可以啊！反正買什麼東西就跟他拿錢，他也不會擋我花」、「如果對方善於理財，我很樂意」。
原PO也提到，有些人在婚前勢必不會答應這種要求，但若戀人結婚，就有一定的保障了，「結婚後會願意讓另一半掌握財政大權嗎？」
▲原PO好奇，大家是否願意將薪水全數交給另一半？（圖／翻攝自PTT）
貼文一出，網友紛紛留言回應，「我爸錢都交給我媽」、「婚前不可能，婚後有商量」、「給他管但不是給他用」、「我直接薪水提款卡給我老婆，自己用炒股的錢當生活費」、「管錢很麻煩好嗎？各自分開比較喜歡」、「不接受，沒結婚就管對方薪水，是控制狂嗎」、「婚後可以溝通，但不會全給，總要留點給自己」、「錢給另一半管就沒有AA問題了」、「可能會設共同帳戶吧」。
還有網友提到1個關鍵條件，「看對方理財的績效，績效不好或是安全感這種理由的，打死不可能給」、「投資計畫或績效報表拿出來看再說」、「誰會理財就誰管」、「男友願意幫我理財，我很樂意欸」、「能夠理財做生意，有能力運用資本，就可以給他啊」、「對

記者姚惠茹／台北報導
和碩（4938）今年為了配合政府防疫政策，跟進科技大廠取消大型尾牙活動，但為了犒賞員工一整年的辛勞，年終尾牙預計每人將發放1萬5千元的紅包，最大獎再抽價值百萬元的特斯拉Model 3，總獎金高達1.2億元。
科技大廠今年相繼取消大型尾牙活動，但是抽獎、紅包樣樣不縮水，像和碩就比照去年的歷來最高規格，首先所有員工都可以拿到「人人有獎」的紅包1萬5千元，再來就是抽獎的最大獎要送出價值百萬元的特斯拉Model 3。
和碩2020年受惠遠距需求帶動筆電、智慧裝置出貨暢旺，再加上下半年有PS5和iPhone 12系列挹注營運動能，帶動單月營收連續10個月突破千億元大關，累計2020年總營收來到1.39兆元，年增2.42%，連續3年突破1.3兆元，創歷史新高紀錄。
----------------------------------------------------------------------

[53] 台幣升值壓力！謝金河：台灣停在低端製造「跟中國拼價格」恐遭淘汰

▲謝金河認為，台幣升值不全然是負面的。（圖／記者湯興漢攝）
記者林妤柔／綜合報導
財信傳媒董事長謝金河在臉書以「要當台幣升值的贏家！不要變輸家！」為題表示，今周刊製作一個很完整的封面故事，央行發出求救訊號，央行的維持滙率穩定機制，很可能已經被美國盯上了，不小心就會成為滙率操縱國，「新台幣升值的壓力只會有增無減，那麼大家必須對台幣升值有妥善的因應對策」。
謝金河表示，前台中市副市長張光遙問他，「台中市是工具機產業最重要的聚落，如果台幣一直升值下去，工具機產業怎麼辦？」謝金河指出，其實這是台灣製造業普通面臨的壓力與挑戰，新台幣升值將啓動一輪淘汰賽，這也是台灣產業升級與轉型的關鍵轉捩點，這其中最重要的是產業有沒有高附加價值！
謝金河直言，去年一年，除了新台幣升值，韓元及人民幣升值幅度也和台幣相當，人民幣還略高於新台幣。過去一年，中國大陸的傳統製造業哀號遍野，情況比台灣更嚴重，從毛利率高低就可以看到對滙率升值的抵抗力。
這些年，台灣的產業毛利率都有相當提升，像氣動元件的亞德客毛利率提升至50.8%，股價已衝上千元，中國大陸很多製造業毛利率都在3%以下，滙率升值便招架不住。謝金河認為，在台幣升值壓力下，如果仍停留在低端製造，跟著中國大陸的企業拼價格，恐怕會淘汰。
台灣企業一定要利用升值機會

▲環球晶圓董事長徐秀蘭。（圖／資料照）
記者周康玉／台北報導
環球晶（6488）今（25）日宣布，再次調整收購世創（Siltronic）條件，將原先預計以收購65%股份就算達成，改為50%；且原先設定收購至德國時間1月27日24時截止，也延後兩週於2月10日截止。
環球晶為收購世創，不僅兩度提高收購價格，今日更將收購最低成就條件放寬至50%，並且時間延長兩星期至2月10日。
環球晶1月22日首先將收購價從每股125歐元，提高至140歐元，又於23日提出最終收購價每股145歐元。環球晶表示，自收購價上調後，不包含公開收購已應賣股份，環球晶已有世創股權比例增至6.06%。目前，參與公開收購已應賣之股份為30.89%。
根據今日調降最低收購股權比例，環球晶仍需取得13.05%股份以達成最低收購股權比例門檻。Siltronic執行董事會於1月23日回應其樂見調升收購價格之舉，並表達該出價具有相當吸引力。
環球晶表示，如未取得最低收購股權比例，將不會對世創提出再次收購要約，而將執行公司已規劃中之其他拓展替代方案。
環球晶重申，無需透過控股或損益表轉移協議（DPLTA）實現本次交易之綜效；環球晶進一步確認自今日起算至少三年內，將不會尋求簽訂DPLTA。
環球晶董事長徐秀蘭表示，透過今日公告，希望清楚表明環球晶立場，並非常鼓勵所有Siltronic股東以最優且最終收購價應賣持股。最低收購股權比例門檻調降後公開收購期間也將延長，提供Siltronic股東更多時間參與應賣。
徐秀蘭說：「我們可以確認，若無法取得最低收購股權比例，我們將立即進行目前已規劃中之其他擴張方案，並不會考慮對世創展開再次收購要約。我們一直以來都聲明無需透過簽訂DPLTA來實現併購綜效，並確認從即日起至少三年內不會尋求DPLTA。」
----------------------------------------------------------------------

[64] 玉山、飛鼠立方衛星首次通聯未果　今晚再次嘗試聯繫

▲立方衛星「飛鼠」和「玉山」搭乘SpaceX發射成功。（圖／翻攝自太空中心、科技部直播）
記者周康玉／台北報導
「玉山」、「飛鼠」立方衛星於台灣時間1月24日晚間11時，順利於美國佛羅里達州卡納維爾角空軍基地發射升空，原訂25日上午10時05分與國家太空中心的玉山地面站及中央大學的

▲台積電 。（圖／記者施怡妏攝）
記者施怡妏／綜合報導
台灣疫情升溫，台積電今（25）日早盤股價重挫，最低來到626元大跌23元，終場以633元作收、下跌2.47%。有一名網友對於日前台積電不斷高漲的狀況提出，很多人都在爭論台積電回檔與否，樂觀的人則認為可能上看1000，但她好奇表示，「萬一崩盤，其他個股也會跟著崩吧？」
該名網友在PTT版上發文，台積電近來氣勢高漲，上週四一度飆至673元，如今卻掉到633元，一堆人都在討論回檔與否，樂觀一點的人則認為上看1000元。她提出，「台積電當了幾個月的送分題，如果真的崩，資金應該也不會回流其他股，反而會因為很多融資買的，要賣其他中小型股去補，結果就是真的台股大逃殺，這樣理解有問題嗎？」
貼文一出後，也網友認同原PO的說法，「台積電崩了就砍其他股逢低買進台積電」、「台積電大跌跟大漲，其他股都會慘」「台積電慢慢回檔的話，資金反而會來到中小型，不過是崩的話，是全部崩」、「台積電就是台股指標，台積電若倒，別股不會好」。
▲有網友指出，「上週五的股票走勢已經給出答案了」。（圖／翻攝自PTT）
也有其他網友指出，「先定義多少叫做崩，不要整天嘴砲」、「外資不就給你看，上星期五賣台積買長榮」、「上周五台積電跌24元，其他股票大漲」、「沒人要台積崩啊，暫時休息讓其他個股發揮，台股多頭走久一點不好嗎」、「不會吧，又不是只有台股能買，要有全球思維」、「只是資金轉移而已，外資出貨跑去買別股而已」。
台股今 (21) 日盤中漲點超過300點，再次站上萬六，終場收在16153點，上漲347點或2.2%，成交量3552.57億元。台積電終場以633元作收、下跌2.47%；鴻海（2317）終場以123元作收、上漲1.23%；股王大立光（3008）以3000元作收、上漲2.04%。其他大型權值除了台積電、聯發科下跌，其他都上漲。
----------------------------------------------------------------------

[77] 連11紅！2020年工業與製造業生產指數「雙創歷年新高」

▲經濟部今（25）日公佈2020年工業生產指數115.84及製造業生產指數116.83，雙雙創下歷年新高紀錄。（圖／經濟部提供）
記者林淑慧／台北報導
經濟部統計處今（25）日公布12月工業生產指數為129.07，為連1

▲2020年11、12月統一發票千萬元特別獎獎號今(25)日公佈，全聯福利中心公佈幸運中獎的消費資訊。圖為示意圖。（圖／全聯提供）
記者林淑慧／台北報導
2020年11、12月統一發票千萬元特別獎獎號今(25)日公佈，全台超市龍頭、全聯福利中心表示，有民眾於樹林中山花費499元購買各式食材，就被幸運之神眷顧中了1000萬特別獎。
2020年11、12月統一發票中獎號碼，今天下午1時30分在東森財經台直播開獎。中獎號碼如下，特別獎（新台幣1000萬元）77815838；特獎（200萬元）39993297；頭獎（20萬元）3組分別為59028801、02813820、06896234。另外，增開六獎2組，末3碼分別為011、427。
▲109年11、12月統一發票中獎號碼。（圖／記者周偉承製）
全聯福利中心本期統一發票共開出一張千萬特別獎與一張兩百萬特獎，其中千萬特別獎開出的幸運門市為樹林中山，消費者花費499元購買珍香豬五花肉條、雞胸肉(去皮)、吳郭魚(大)、老鷹紅豆麻糬、義美芝麻湯圓、桂冠花生湯圓、紀文小魚卵捲、青蔥(田尾)、香草園洗選蛋等9樣商品，即中了1000萬特別獎。
而兩百萬特獎開出門市為中山興安，消費者花費281元購買鴻喜菇與餅乾等商品，就被幸運神眷顧。全聯11、12月份共開出超過37萬個獎項，提醒11、12月份有來全聯消費的民眾，不要忘記對一下手中的發票，千萬特別獎與兩百萬特獎得主可能就是自己。
----------------------------------------------------------------------

[90] 快訊／小七門市買25元茶飲中大獎！統一超商發票特別獎得主出爐

▲2020年11、12月統一發票千萬元特別獎獎號今(25)日公佈，統一超商公佈幸運中獎者的消費資訊。圖為示意圖。（圖／統一超商提供）
記者林淑慧／台北報導
2020年11、12月統一發票千萬元特別獎獎號今(25)日公佈，本期千萬特別獎號碼是「77815838」，超商龍頭統一超商表示，本期共有二位消費者獲得千萬大獎，一人在台北市大安區恆安門市共購買155元的民生用品、CITY CAFE及大亨堡；一人在台北市中山區松怡門市購買25元茶飲，幸運對中大獎。
2020年11、12月統一發票中獎號碼，今天下午1時30分在東森財經台直播開獎。中獎號碼如下，特別獎

In [ ]:
browser.quit()